# ENGR 240 - Worksheet 4.2: LU Factorization Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%204/LU_Factorization_Demo.ipynb)

This notebook demonstrates the computational efficiency gained by using LU factorization when solving multiple linear systems with the same coefficient matrix but different right-hand sides. This is based on Task 3 from Worksheet 4.2.

## Problem Statement

Given the linear system:

$$\begin{bmatrix} 2 & -1 & 5 \\ 3 & 2 & 1 \\ 1 & -4 & 2 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} k \\ 5 \\ -4 \end{bmatrix}$$

Solve for the values of the $x$ vector that correspond to $k$ values ranging from -5 to 5 in increments of 0.0001. 

We'll compare two approaches:
1. Solving each system directly 
2. Using LU factorization to decompose the matrix once, then solving for each right-hand side

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.linalg

## Define the Problem

First, we'll define our coefficient matrix A and create an array of k values.

In [ ]:
# Define the coefficient matrix A (same as in the worksheet)
A = np.array([
    [2, -1, 5],
    [3, 2, 1],
    [1, -4, 2]
])

# Create a range of k values from -5 to 5 with small increments
k = np.arange(-5, 5.0001, 0.0001)
print(f"Matrix A:\n{A}")
print(f"Number of k values to process: {len(k)}")

## Method 1: Solving Without LU Factorization

In this approach, we'll solve the entire system for each value of k.

In [ ]:
# Without LU Factorization
start_time = time.time()
xk = np.ones((len(k), 3))  # preallocate space for solutions

for ndx in range(len(k)):
    # Build RHS vector using the current value of k
    b = np.array([k[ndx], 5, -4])
    
    # Solve the system for this RHS vector
    x = np.linalg.solve(A, b)
    
    # Store the solution in the ndx row of xk
    xk[ndx, :] = x

direct_solve_time = time.time() - start_time
print(f"Time taken without LU factorization: {direct_solve_time:.4f} seconds")

## Method 2: Solving With LU Factorization

In this approach, we'll use SciPy's optimized LU factorization implementation.

In [ ]:
# With LU Factorization using SciPy's method
start_time = time.time()
xk_LU = np.ones((len(k), 3))  # preallocate space for solutions

# Compute LU factorization of A using SciPy's method
lu, piv = scipy.linalg.lu_factor(A)

for ndx in range(len(k)):
    # Build RHS vector using the current value of k
    b = np.array([k[ndx], 5, -4])
    
    # Solve the system using the factorized matrix
    x = scipy.linalg.lu_solve((lu, piv), b)
    
    # Store the solution
    xk_LU[ndx, :] = x

lu_solve_time = time.time() - start_time
print(f"Time taken with LU factorization: {lu_solve_time:.4f} seconds")

## Comparison and Performance Analysis

In [ ]:
print(f"Direct solve time:  {direct_solve_time:.4f} seconds")
print(f"LU solve time:      {lu_solve_time:.4f} seconds")

print(f"\nSpeed improvement with LU: {direct_solve_time/lu_solve_time:.2f}x faster")

# Verify that solutions match
print(f"\nMaximum difference between direct and LU solutions: {np.max(np.abs(xk - xk_LU)):.2e}")

## Visualizing the Solutions

Let's visualize how the solution components ($x_1$, $x_2$, $x_3$) change with the parameter $k$:

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(k, xk[:, 0], label='$x_1$')
plt.plot(k, xk[:, 1], label='$x_2$')
plt.plot(k, xk[:, 2], label='$x_3$')
plt.grid(True)
plt.xlabel('k value')
plt.ylabel('Solution components')
plt.title('Solution components vs. parameter k')
plt.legend()
plt.xlim(-5, 5)
plt.show()

## Discussion

### Computational Efficiency

The results demonstrate the efficiency of using LU factorization when solving multiple linear systems with the same coefficient matrix but different right-hand sides. Here's why this happens:

1. **Direct Solve Method**: For each new right-hand side, we perform full Gaussian elimination (an O(n³) operation for an n×n matrix).

2. **LU Factorization Method**: We perform LU factorization once (an O(n³) operation), but for each new right-hand side, we only need to perform forward and backward substitution (each an O(n²) operation).

When solving many systems, the factorization cost is amortized over all the systems, making the LU method more efficient.

### Math Behind LU Factorization

LU factorization decomposes a matrix A into a product of a lower triangular matrix L and an upper triangular matrix U:

$$A = LU$$

When partial pivoting is used (which is typical for numerical stability), we have:

$$PA = LU$$

where P is a permutation matrix.

To solve the system $Ax = b$, we:

1. Compute the LU factorization $PA = LU$
2. Solve $Ly = Pb$ using forward substitution
3. Solve $Ux = y$ using backward substitution

### Applications

This technique is valuable in many engineering applications, such as:

- Finite element analysis with multiple load cases
- Circuit analysis with multiple sources
- Parameter sensitivity studies (as demonstrated in this worksheet)
- Control system analysis with varying inputs